## Import Qiskit libraries:

In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, BasicAer, Aer, execute
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit.utils import algorithm_globals
from qiskit.utils import QuantumInstance

from qiskit.visualization.utils import (_bloch_multivector_data,_paulivec_data,matplotlib_close_if_inline,)
from qiskit.visualization import plot_bloch_multivector, plot_state_qsphere, plot_state_city, plot_bloch_vector
from qiskit.compiler import assemble
from qiskit.circuit.library.standard_gates import (IGate, U1Gate,XGate,YGate,ZGate,HGate,SGate,SdgGate,TGate,TdgGate,RXGate,RYGate,RZGate)

from matplotlib import pyplot as plt

import numpy as np
import time
import random
import sys


#from azure.quantum.qiskit import AzureQuantumProvider
from qiskit.quantum_info import Statevector
from qiskit.extensions import Initialize


from PIL import Image
import sys
from IPython.display import clear_output
import time 
from tqdm import * 

## Level-1: Spacecraft

In [2]:
import time
import random
import sys

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, BasicAer, Aer, execute
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
#from azure.quantum.qiskit import AzureQuantumProvider
from qiskit.utils import algorithm_globals
from qiskit.utils import QuantumInstance
from qiskit.visualization import plot_bloch_multivector
from qiskit.quantum_info import Statevector
from qiskit.extensions import Initialize

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
from IPython.display import clear_output
import time 
from tqdm import * 


#backend = provider.get_backend("ionq.simulator")
backend = BasicAer.get_backend('qasm_simulator')
sim = Aer.get_backend('aer_simulator')

# just of effects. add a delay of 1 second before performing any action
SLEEP_BETWEEN_ACTIONS = 0.01
SCORE = 0

single_q_gates = ['h', 'x', 'y', 'I', 'z', 's', 'sdg', 't', 'tdg'] 
single_q_param_gates = ['rx', 'ry', 'rz', 'u', 'p']
double_q_gates = ['cx', 'cz', 'cy', 'ch', 'swap']
double_q_param_gates = ['cu', 'cp', 'crx', 'cry', 'crz'] 

ket_0 = np.array([[1],[0]])
iota = 1j
def u3_mat(theta, phi, lambda_v):
    matrix_gen = np.array(([np.cos(theta/2), -(np.exp(iota*lambda_v))*np.sin(theta/2)],[(np.exp(iota*phi))*np.sin(theta/2), (np.exp(iota*(phi+lambda_v)))*np.cos(theta/2)]))
    return matrix_gen

def welcome_msg():
    msg = """
    Welcome to the Oggy and Cockroaches game
    Version: 1.0.0
    Developed by: Utkarsh Singh
    
    Rules:
      1. There are a totatl of 3 levels in the game. NOTE: I have not added the 4th one yet. But that one is also ready. I'll integrate it in the near future. 
      2. To win the game you must complete all the levels. 
      3. Press enter to board the spacecraft.
    
    """
    print(msg)
    
def my_float(s):
    constants = {"pi": 3.14159, "e": 2.71928}
    if s in constants:
        return constants[s]
    else:
        return float(s)

def draw_img(adrs):
    ImageItself = Image.open(adrs)
    #ImageNumpyFormat = np.asarray(ImageItself)
    plt.imshow(ImageItself)
    plt.show(block = False)
    plt.pause(1) # pause how many seconds
    plt.close('all')
    
def video(image):
    # intro = ['O&C/1.png', 'O&C/2.png', 'O&C/3.png']
    for i in tqdm(range(len(image)), leave=False):
        clear_output(wait=True)
        draw_img(image[i])
        time.sleep(3)
#draw_img('O&C/1.png')

def eng_circuit(theta, phi, lamb, g):
    q =QuantumRegister(1, 'q')
    c = ClassicalRegister(1, 'c')
    qc = QuantumCircuit(q,c)
    state = u3_mat(theta, phi, lamb).dot(ket_0)
    psi = Statevector(state)
    init_gate = Initialize(psi)
    init_gate.label = "code"
    qc.append(init_gate, [0])
    
    for i in range(len(g)):
        if type(g[i]) is tuple:
            
            gate = g[i][0]
            param = float(g[i][1])
            if gate == 'rx':
                qc.rx(param , [0]) 
            elif gate == 'ry':
                qc.ry(param, [0])
            elif gate == 'rz':
                qc.rz(param, [0])
            elif gate == 'u':
                qc.u(param, [0])
            elif gate == 'p':
                qc.p(param, [0])
            else:
                pass
        else:
            if g[i] == 'h':
                qc.h([0])
            elif g[i] == 'x':
                qc.x([0])
            elif g[i] == 'y':
                qc.y([0])
            elif g[i] == 'z':
                qc.z([0])
            elif g[i] == 'i':
                qc.barrier()
            elif g[i] == 't':
                qc.t([0])
            elif g[i] == 'tdg':
                qc.tdg([0])
            elif g[i] == 's':
                qc.s([0])
            elif g[i] == 'sdg':
                qc.sdg([0])
            else:
                pass
    qc.measure([0],[0])
    # sim = Aer.get_backend('aer_simulator')
    qc.save_statevector()
    res = sim.run(qc, shots = 1000).result().get_counts()
    return res


def eng_gate_op():
    # print("Enter the gate(s) to reset the first Qubit.")
    g1 = input("Apply first Quantum gate").lower()
    if g1 in single_q_param_gates:
        param = my_float(input("Parameter value ="))
        g1 = (g1, param)
    else:
        pass 
    
    in1 = input("Do you want to apply more gates? (Y/N)").upper()
    if in1 == 'Y':
        g2 = input("Apply 2nd quantum operation:").lower()
        if g2 in single_q_param_gates:
            param2 = my_float(input("Parameter value ="))
            g2 = (g2, param2)
        else:
            pass 
    else:
        g2 = 'I'
        
    in2 = input("Do you want to apply more gates? (Y/N)").upper()
    if in2 == 'Y':
        g3 = input("Apply last quantum operation:").lower()
        if g3 in single_q_param_gates:
            param3 = my_float(input("Parameter value ="))
            g3 = (g3, param3)
        else:
            pass 
    else:
        g3 = 'I'
    return [g1, g2, g3]

def level_one(gate1, gate2, gate3, score, level):
    counts1 = eng_circuit(np.pi/4,3*np.pi/4,0, gate1) #initial state of the qubit1: u3(np.pi/4, 3*np.pi/4, 0)
    counts2 = eng_circuit(np.pi/2,np.pi/2,0, gate2) #initial state of the qubit2: u3(np.pi/2, np.pi/2, 0)
    counts3 = eng_circuit(np.pi/2,3*np.pi/2,0, gate3) #initial state of the qubit3: u3(np.pi/2, 3*np.pi/2, 0)
    key_counts1 = [ key1 for key1 in counts1 if counts1[key1] >40]
    key_counts2 = [ key2 for key2 in counts2 if counts2[key2] >40]
    key_counts3 = [ key3 for key3 in counts3 if counts3[key3] >40]
    key_counts = key_counts1 + key_counts2 + key_counts3
    #SCORE = 0
    if '1' in key_counts:
        #score = score
        print(counts1, counts2, counts3)
        print("Game Over")
    else:
        level = level+1
        score = score + 100
        print('Engine is ready to launch!')


/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/opt/conda/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 f

## Level-2: Airlock

In [3]:
qa =QuantumRegister(3, 'qa')
ca = ClassicalRegister(3, 'ca')
al = QuantumCircuit(qa,ca)

for i in range(3):
    al.h([i])


def airlock_op():
    air1 = input("Apply Quantum operation").lower()
    air_in1 = air1
    if air1 in double_q_param_gates:
        param = my_float(input("Parameter value ="))
        cq = int(input("Control Qubit:"))
        tq = int(input("Target Qubit:"))
        if cq or tq>2:
            print("Qubit index out of range")
        else:
            air_in1 = [(air1, param), cq, tq]
    elif air1 in single_q_param_gates:
        param = my_float(input("Parameter value ="))
        q = int(input("Qubit:"))
        if q>2:
            print("Qubit index out of range")
        else:
            air_in1 = [(air1, param), q]
    elif air1 in double_q_gates:
        cq = int(input("Control Qubit:"))
        tq = int(input("Target Qubit:"))
        if cq or tq >2:
            print("Qubit index out of range")
        else:
            air_in1 = [air1, cq, tq]
    elif air1 in single_q_gates:
        q = int(input("Qubit:"))
        if q>2:
            print("Qubit index out of range")
        else:
            air_in1 = [air1, q]
    else:
        print("Invalid entry")
        
    return air_in1
        
        
# double_q_gates = ['cx', 'cz', 'cy', 'ch', 'swap']
# double_q_param_gates = ['cu', 'cp', 'crx', 'cry', 'crz']   
    
def airlock_circuit(air_input):
    qa =QuantumRegister(3, 'qa')
    ca = ClassicalRegister(3, 'ca')
    al = QuantumCircuit(qa,ca)
    ## input forms: [gate, qubit], [(gate, param), qubit], [c_gate, qubit, qubit], and [(c_gate, param), qubit, qubit]
    
    if len(air_input)==2:
        if type(air_input[0]) is tuple:
            air_gate = air_input[0][0]
            param = float(air_input[0][1])
            qubit = int(air_input[1])
            if air_gate == 'rx':
                al.rx(param , [qubit]) 
            elif air_gate == 'ry':
                al.ry(param, [qubit])
            elif air_gate == 'rz':
                al.rz(param, [qubit])
            elif air_gate == 'u':
                al.u(param, [0])
            elif air_gate == 'p':
                al.p(param, [qubit])
            else:
                pass
        else:

            air_gate = air_input[0]
            qubit = air_input[1]
            if air_gate == 'h':
                al.h([qubit])
            elif air_gate == 'x':
                al.x([qubit])
            elif air_gate == 'y':
                al.y([qubit])
            elif air_gate == 'z':
                al.z([qubit])
            elif air_gate == 'i':
                al.barrier()
            elif air_gate == 't':
                al.t([qubit])
            elif air_gate == 'tdg':
                al.tdg([qubit])
            elif air_gate == 's':
                al.s([qubit])
            elif air_gate == 'sdg':
                al.sdg([qubit])
            else:
                pass

    elif len(air_input) == 3:
        if type(air_input[0]) is tuple:
            air_cgate = air_input[0][0]
            cparam = float(air_input[0][1])
            cqubit = int(air_input[1])
            tqubit = int(air_input[2])
            if air_cgate == 'cu':
                al.cu(cparam, 0, 0, 0, cqubit, tqubit) 
            elif air_cgate == 'cp':
                al.cp(cparam, cqubit, tqubit)
            elif air_cgate == 'crz':
                al.crz(cparam, cqubit, tqubit)
            elif air_cgate == 'cry':
                al.cry(cparam, cqubit, tqubit)
            elif air_cgate == 'crx':
                al.crx(cparam, cqubit, tqubit)
            else:
                pass
        else:
            air_cgate = air_input[0]
            cqubit = air_input[1]
            tqubit = air_input[2]
            if air_cgate == 'ch':
                al.ch(cqubit, tqubit)
            elif air_cgate == 'cx':
                al.cx(cqubit, tqubit)
            elif air_cgate == 'cy':
                al.cy(cqubit, tqubit)
            elif air_cgate == 'cz':
                al.cz(cqubit, tqubit)
            elif air_cgate == 'swap':
                al.swap(cqubit, tqubit)
            else:
                pass
    else:
        al.barrier()
    return al            


def operations():
    num = int(input("How many quantum operations/gates do you want to use? (Max = 4)"))
    if num == 1:
        first_in = airlock_op()
        circ1 = airlock_circuit(first_in)
        circuit = al.compose(circ1)
        
    elif num == 2:
        in1 = airlock_op()
        circ1 = airlock_circuit(in1)
        al_circ = al.compose(circ1)
        in2 = airlock_op()
        circ2 = airlock_circuit(in2)
        circuit = al_circ.compose(circ2)
        
    elif num == 3:
        in1 = airlock_op()
        circ1 = airlock_circuit(in1)
        al_circ = al.compose(circ1)
        in2 = airlock_op()
        circ2 = airlock_circuit(in2)
        circ_al= al_circ.compose(circ2)
        in3 = airlock_op()
        circ3 = airlock_circuit(in3)
        circuit = circ_al.compose(circ3)
    elif num == 4:
        in1 = airlock_op()
        circ1 = airlock_circuit(in1)
        al_circ = al.compose(circ1)
        in2 = airlock_op()
        circ2 = airlock_circuit(in2)
        circ_al= al_circ.compose(circ2)
        in3 = airlock_op()
        circ3 = airlock_circuit(in3)
        circ3 = circ_al.compose(circ3)
        in4 = airlock_op()
        circ4 = airlock_circuit(in4)
        circuit = circ3.compose(circ4)
    return circuit      
    

def level2(score, level):
    print("Welcome to the Level-2")
    print("Cockroaches have damaged the protection shield of the airlock. Be aware!")
    airlock = ['O&C/airlock/1.png', 'O&C/airlock/2.png', 'O&C/airlock/3.png', 'O&C/airlock/4.png', 'O&C/airlock/5.png', 'O&C/airlock/6.png', 'O&C/airlock/7.png']
    video(airlock) 
    time.sleep(SLEEP_BETWEEN_ACTIONS)
   
    input("Press ENTER to start level-2")
    time.sleep(SLEEP_BETWEEN_ACTIONS)
    print("HINT: All the 3 qubits are in equal supeposition.")
    
    airlock_circuit = operations()
    airlock_circuit.measure([0,1,2],[0,1,2])
    # airlock_circuit.save_statevector()
    airlock_res = execute(airlock_circuit, backend = Aer.get_backend('qasm_simulator'), shots = 1000).result().get_counts()
    out_state = [key for key in airlock_res]
    comp_state = [000, 111]
    
    if out_state != comp_state:
        print("Game Over")
    else:
        level = level + 1
        score = score + 100
        print("Level Complete!\nYou are in outer space.")
        print("Your score is:",score)

## Level-3:

In [4]:
def choose_gate(message):
    print("Choose get sequence for ", message)
    input1 = input("Enter the 1st gate that you want to apply on your qubit:").lower()
    in2 = input("Do you want to apply a second gate? (Y/N)").upper()
    if in2 == 'Y':
        input2 = input("Eneter the 2nd gate that you want to apply on your qubit:").lower()
        gate_seq = [input1, input2]
    else:
        gate_seq = [input1, 'i']         
    return gate_seq

def teleportation(message):
    if message == '11':
        required_seq = ['z', 'x']
    elif message == '10':
        required_seq = ['z', 'i']
    elif message == '01':
        required_seq = ['x', 'i']
    else:
        required_seq = ['i', 'i']
    return required_seq
        
Jack_message = [ '11', '01',   '00', '01', '00',   '11', '10',  '00', '01', '11']  



def win_check(input, required):
    if input == required:
        print("Nice Job!\nYou have decoded this one.")
    else:
        print("Better luck next time.")
        sys.exit()


def level3():
    print("Welcome to level-3")
    teleport = ['O&C/teleportation/8.png', 'O&C/teleportation/9.png', 'O&C/teleportation/10.png', 'O&C/teleportation/11.png', 'O&C/teleportation/12.png', 'O&C/teleportation/13.png', 'O&C/teleportation/14.png', 'O&C/teleportation/15.png', 'O&C/teleportation/16.png', 'O&C/teleportation/17.png', 'O&C/teleportation/18.png']
    video(teleport)
    time.sleep(SLEEP_BETWEEN_ACTIONS)
    print("To complete this level, you must apply the correct sequence of gates on your qubit.")
    print("Hint: You have Jack's measurement results. [ 11 01   00 01 00   11 10   00 01 11]")
    time.sleep(SLEEP_BETWEEN_ACTIONS)
    input("Hit ENTER to start the game.")
    for mes in Jack_message:
        time.sleep(SLEEP_BETWEEN_ACTIONS)
        ing = choose_gate(mes)
        req = teleportation(mes) 
        win_check(ing, req)
    

In [5]:
def start():
    #time.sleep(SLEEP_BETWEEN_ACTIONS)
    # draw_img('O&C/1.png')
    # time.sleep(SLEEP_BETWEEN_ACTIONS)
    # draw_img('O&C/2.png')
    # time.sleep(SLEEP_BETWEEN_ACTIONS)
    # draw_img('O&C/3.png')
    # time.sleep(SLEEP_BETWEEN_ACTIONS)
    level = 1
    while True:
        intro = ['O&C/1.png', 'O&C/2.png', 'O&C/3.png']
        video(intro)

        input("Press Enter to play the game")
        welcome_msg()
        time.sleep(SLEEP_BETWEEN_ACTIONS)
        draw_img('O&C/gif1.gif')
        draw_img('O&C/eng.png')    
        #time.sleep(SLEEP_BETWEEN_ACTIONS)
        print("Enter the gate(s) to reset the first (leftmost) Qubit.")
        gate1 = eng_gate_op()
        time.sleep(SLEEP_BETWEEN_ACTIONS)
        print("Enter the gate(s) to reset the second Qubit.")
        gate2 = eng_gate_op()
        time.sleep(SLEEP_BETWEEN_ACTIONS)
        print("Enter the gate(s) to reset the third Qubit.")
        gate3 = eng_gate_op()
        level_one(gate1, gate2, gate3, SCORE, level)
        time.sleep(SLEEP_BETWEEN_ACTIONS)

        if level == 2:
            print("Welcome to the Level-2")
            print("Cockroaches have damaged the protection shield of the airlock. Be aware!")
            airlock = ['O&C/airlock/1.png', 'O&C/airlock/2.png', 'O&C/airlock/3.png', 'O&C/airlock/4.png', 'O&C/airlock/5.png', 'O&C/airlock/6.png', 'O&C/airlock/7.png']
            video(airlock) 
            time.sleep(SLEEP_BETWEEN_ACTIONS)
            level2(SCORE, level )
            if level == 3:
                level3()
                
            
        else:
            print("Please complete level 1")

if __name__ == "__main__":
    start()